In [5]:
from transformers import AutoTokenizer
import pprint
from collections import Counter
import numpy as np

In [2]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokens = tokenizer("Hello world")["input_ids"]
print(tokens)

[15496, 995]


In [21]:
# pprint.pprint(vars(tokenizer))
# print(tokenizer.get_vocab())

# Print the counts of tokens in the vocabulary
counts = Counter(tokenizer.get_vocab())
print('len(counts)', len(counts))
# print(counts)

print(tokenizer.vocab_size)
print(type(tokenizer.vocab))
# pprint.pp(tokenizer.vocab)

len(counts) 50257
50257
<class 'dict'>


In [22]:
# sorted_token_freqs = np.array([item[1] for item in counter.most_common()])
# print(sorted_token_freqs.shape)

# sorted_token_ids = np.arange(len(sorted_token_freqs))
# print(sorted_token_ids[:10])

# import matplotlib.pyplot as plt
# plt.plot(sorted_token_ids, sorted_token_freqs)

In [ ]:
# Plot raw freqs against token id
